<a href="https://colab.research.google.com/github/robocarstore/donkey-car-training-on-google-colab/blob/master/Donkey_Car_Training_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU allocation
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [0]:
# !pip uninstall tensorflow

In [0]:
# !pip install tensorflow-gpu==1.13.1

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Git Clone the donkey repository

In [0]:
!git clone https://github.com/autorope/donkeycar.git 
%cd /content/donkeycar

Cloning into 'donkeycar'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 10609 (delta 6), reused 10 (delta 3), pack-reused 10581
Receiving objects: 100% (10609/10609), 58.76 MiB | 10.34 MiB/s, done.
Resolving deltas: 100% (6546/6546), done.
/content/donkeycar
Already on 'dev'
Your branch is up to date with 'origin/dev'.


## Install donkey car

In [0]:
!pip3 install -e .[pc]

Obtaining file:///content/donkeycar
     |████████████████████████████████| 92kB 35.9MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.4.0-cp36-none-any.whl size=48331 sha256=d5ba96bc2a1733dc43fcbbf0f2f4a60fab7c130b6046a347a134c42e0094e05a
  Stored in directory: /root/.cache/pip/wheels/82/e5/de/d90d0f397648a1b58ffeea1b5742ac8c77f71fd43b550fa5a5
Successfully built paho-mqtt
  Running setup.py develop for donkeycar


## Create Project

In [0]:
!donkey createcar --path /content/mycar

using donkey v3.1.0 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: complete
Copying car config defaults. Adjust these before starting your car.
Copying train script. Adjust these before starting your car.
Copying my car config overrides
Donkey setup complete.


## Prepare Data
### Copy the following code and run on pi


```bash
cd ~/mycar/data
tar -czf tub_xx_yyyy_mm_dd.tar.gz tub_xx_yyyy_mm_dd

```

This will create a tub_xx_yyyy_mm_dd.tar.gz file under ~/mycar/data

## Upload Data


### Copy the tub to your local pc

Run this on your local pc if you are using linux/mac
```
sftp pi@raspberry.local
cd ~/mycar/data
get tub_xx_yyyy_mm_dd.tar.gz
```

If you are on a windows, download sftp utility like [filezilla](https://filezilla-project.org/) or [putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)

### Define your tub name here

In [0]:
tub_name="tub_8_19-08-23"

### Upload the tub to Google Drive

First upload the tub_x_yyyy_mm_dd.tar.gz to Google Drive. We will then mount Google Drive from colab and copy the data from Drive directly. When you run the cell below, you will need to click the link and generate an authorization code to for colab to access your drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Suppose you upload the tub_xx_yyyy_mm_dd.tar.gz to Google Drive/mycar/tub_xx_yyyy_mm_dd.tar.gz, this is how you copy it from Google Drive to colab

In [0]:
%cd /content/mycar/data
!cp /content/drive/My\ Drive/mycar/{tub_name}.tar.gz .

/content/mycar/data


And untar it to the right place

In [0]:
!tar -xzf {tub_name}.tar.gz

Check whether the data is there

In [0]:
!ls {tub_name}

1000_cam-image_array_.jpg  3911_cam-image_array_.jpg  record_2457.json
1001_cam-image_array_.jpg  3912_cam-image_array_.jpg  record_2458.json
1002_cam-image_array_.jpg  3913_cam-image_array_.jpg  record_2459.json
1003_cam-image_array_.jpg  3914_cam-image_array_.jpg  record_245.json
1004_cam-image_array_.jpg  3915_cam-image_array_.jpg  record_2460.json
1005_cam-image_array_.jpg  3916_cam-image_array_.jpg  record_2461.json
1006_cam-image_array_.jpg  3917_cam-image_array_.jpg  record_2462.json
1007_cam-image_array_.jpg  3918_cam-image_array_.jpg  record_2463.json
1008_cam-image_array_.jpg  3919_cam-image_array_.jpg  record_2464.json
1009_cam-image_array_.jpg  391_cam-image_array_.jpg   record_2465.json
100_cam-image_array_.jpg   3920_cam-image_array_.jpg  record_2466.json
1010_cam-image_array_.jpg  3921_cam-image_array_.jpg  record_2467.json
1011_cam-image_array_.jpg  3922_cam-image_array_.jpg  record_2468.json
1012_cam-image_array_.jpg  3923_cam-image_array_.jpg  record_2469.json
1013_ca

## Train your model

In [0]:
!python /content/mycar/manage.py train --type linear --model /content/mycar/models/mypilot.h5

using donkey v3.1.0 ...
loading config file: /content/mycar/config.py
loading personal config over-rides

config loaded
W0830 00:46:43.192679 140405366081408 deprecation_wrapper.py:119] From /content/donkeycar/donkeycar/parts/keras.py:18: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 00:46:43.192966 140405366081408 deprecation_wrapper.py:119] From /content/donkeycar/donkeycar/parts/keras.py:18: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-08-30 00:46:43.209827: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-08-30 00:46:43.210044: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x25ab2c0 executing computations on platform Host. Devices:
2019-08-30 00:46:43.210072: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-08-30 00:46:43.211978: I tensorflow/stream_executor/platform/default/dso_loade

Check if the model is generated



In [0]:
!ls -alh /content/mycar/models

total 3.2M
drwxr-xr-x 2 root root 4.0K Aug 30 00:47 .
drwxr-xr-x 6 root root 4.0K Aug 30 00:46 ..
-rw-r--r-- 1 root root 3.2M Aug 30 00:47 mypilot.h5
-rw-r--r-- 1 root root  24K Aug 30 00:47 mypilot.h5_loss_acc_0.111756.png


In [0]:
# from IPython.display import Image
# Image('models/mypilot.h5_loss_acc_0.086745.png')

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [0]:
from google.colab import files

%cd /content/mycar/models
!ls -ahl
files.download('mypilot.h5')

/content/mycar/models
total 3.2M
drwxr-xr-x 2 root root 4.0K Aug 30 00:47 .
drwxr-xr-x 6 root root 4.0K Aug 30 00:46 ..
-rw-r--r-- 1 root root 3.2M Aug 30 00:47 mypilot.h5
-rw-r--r-- 1 root root  24K Aug 30 00:47 mypilot.h5_loss_acc_0.111756.png


MessageError: ignored

Alternatively, you can copy the model back to Google Drive too

In [0]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models/mypilot.h5

### Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

```
sftp pi@raspberry.local
cd mycar/models
put mypilot.h5
```

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Bonus - Salient Object Visualization

In [0]:
# !pip install git+https://github.com/autorope/keras-vis.git
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis
  
  

Uninstalling keras-vis-0.5.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras_vis-0.5.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/vis/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/structure.py
Proceed (y/n)? y
  Successfully uninstalled keras-vis-0.5.0
  Cloning https://github.com/sctse999/keras-vis to /tmp/pip-req-build-5cv1exnd
  Running command git clone -q https://github.com/sctse999/keras-vis /tmp/pip-req-build-5cv1exnd
  Created wheel for keras-vis: filename=keras_vis-0.5.0-py2.py3-none-any.whl size=38990 sha256=d4e7dfb0630b25cdce84566036320950f747369658106e83e97fbbbb1d9ac6df
  Stored in directory: /tmp/pip-ephem-wheel-cache-kgtgkatf/wheels/29/87/8e/abd2257f08391eabe7552711aecf08cbb50f79877210b21be0
Successfully built keras-vis


In [0]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/mypilot.h5 --type linear --salient

/content/mycar
using donkey v3.1.0 ...
loading config file: ./config.py
loading personal config over-rides

config loaded
W0830 00:52:41.127935 139854661654400 deprecation_wrapper.py:119] From /content/donkeycar/donkeycar/parts/keras.py:18: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 00:52:41.128137 139854661654400 deprecation_wrapper.py:119] From /content/donkeycar/donkeycar/parts/keras.py:18: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-08-30 00:52:41.142732: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-08-30 00:52:41.142952: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x329ea00 executing computations on platform Host. Devices:
2019-08-30 00:52:41.142979: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-08-30 00:52:41.144832: I tensorflow/stream_executor/platform/default/dso_loa